## Семинар 6: ArgParse. JSON. YAML. pydantic.

## Разминка ([выражения генераторы](https://www.python.org/dev/peps/pep-0289/#the-details)) 


In [ ]:
# генератор
a = [index for index in range(10)]
print(a)

In [ ]:
# выражение генератор (generator expression)
a = (index for index in range(10))
print(a)
for value in a:
    print(value, end=', ')

In [ ]:
a = [1, 2, 3]
b = [10, 20, 30]
gen = (x + y for x in a for y in b)
print(type(gen))
a = [4, 5, 6]
b = [400, 500, 600]
print(list(gen))

Что будет напечатано в результате?

### ArgParse

Модуль argparse упрощает написание интерфейсов командной строки.

Программа определяет, какие аргументы ей требуются, а argparse выясняет, как их анализировать, используя sys.argv. Модуль argparse также автоматически генерирует справочные сообщения и сообщает об ошибках, когда пользователи вводят недопустимые аргументы.

```python
parser = ArgumentParser()

parser.add_argument('--batch_size', default=32, type=int)
parser.add_argument('--hidden_dim', type=int, default=128)
    
args = parser.parse_args()

print(args)
```

Подробнее про определение аргументов: [argument-methods](https://docs.python.org/3/library/argparse.html#the-add-argument-method)

После считывания аргументов без ошибок получим объект типа ```Namespase```:

```python
>>> Namespace(batch_size=32, hidden_dim=128)
```

К его поляем можно обращаться по имени:
```python
print(args.hidden_dim)

>>> 128
```

Как это реализовано? Как посмотреть реализацию чего-либо?

### JSON

JSON (JavaScript Object Notation) - это облегченный формат обмена данными. Людям легко читать и писать(?), а комьютеры лего анализируют и генерируют его. JSON основан на подмножестве стандарта языка программирования JavaScript ECMA-262, 3-е издание - декабрь 1999 г. JSON - это текстовый формат, который полностью не зависит от языка, но использует соглашения, знакомые программистам семейства языков C, включая C, C ++, C #, Java, JavaScript, Perl, Python и многие другие. Эти свойства делают JSON идеальным языком обмена данными.

Сохраним файл config.json со следующим содержанием:
```json
{"batch_size": 32, "hidden_size": 128}
```

Для того, чтобы прочитать файл можно воспользоваться командой ```json.load()```:

```python
import json 

with open('config.json', "r") as f:
    config = json.load(f)
print(config)
```

Пример сохранения данных в JSON:

```python
import json

data = {"accuracy": 0.95}

with open("data.json", "w") as f:
    json.dump(data, f)
```

## PyYaml
Yaml $-$ это удобочитаемый язык сериализации данных. Обычно, для файлов используется формат ```.yaml```. Посмотрим пример, для удобства у нас будет использован не внешний файл, а строка:

In [ ]:
import yaml

document = """
a: 1
b:
    c: 3
    d: 4
e: [1, 2, 3]
f:
    - 1
    - 2
    - 3

g: True
h: hello #hello
# z: 5 <- comment

"""
q = yaml.load(document, Loader=yaml.FullLoader)
print(q)

Если мы работаем с файлами, у нас точно также работают методы ```dump```:

```python
data = {'accuracy': 0.93}

with open('data.yaml', 'w') as f:
    data = yaml.dump(data, f)
```

 и ```load```:

```python
with open("data.yaml", "r") as f:
    data = yaml.safe_load(f)
```

### pydantic

Проверка данных и конфигураций с использованием аннотаций типа Python. Библиотека применяет подсказки типов во время выполнения и предоставляет удобные для пользователя ошибки, когда данные недействительны.

*Define how data should be in pure, canonical python; validate it with pydantic.*

In [ ]:
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel


class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []


external_data = {
    'id': '123',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, '3'],
}

user = User(**external_data)

print(user.id)

Сложные проверки и сложные связи между объектами можно реализовать, используя декоратор ```validator```:

In [ ]:
from pydantic import BaseModel, ValidationError, validator


class UserModel(BaseModel):
    name: str
    username: str
    password1: str
    password2: str

    @validator('name')
    def name_must_contain_space(cls, v):
        if ' ' not in v:
            raise ValueError('must contain a space')
        return v.title()

    @validator('password2')
    def passwords_match(cls, v, values, **kwargs):
        if 'password1' in values and v != values['password1']:
            raise ValueError('passwords do not match')
        return v

    @validator('username')
    def username_alphanumeric(cls, v):
        assert v.isalnum(), 'must be alphanumeric'
        return v


user = UserModel(
    name='samuel colvin',
    username='scolvin',
    password1='zxcvbn',
    password2='zxcvbn',
)

print(user)

Чтобы получить данные в базовом формате – можно использовать метод ```dict```:

In [ ]:
print(user.dict())